# Aposteriori Unimodality applications

## Real life data

### Preprocessing

In [10]:
%load_ext autoreload 
%autoreload 2

import re
import sys
import hashlib

import pandas as pd
import numpy as np
from pathlib import Path


# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
from src import aposteriori

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
df = pd.read_pickle("../data/attitudes_embedded.csv")
df

tweet               toyou  \
0    " you ugly bitch!!!"- that seems to be my real...        [2, 1, 1, 2]   
1    """I am happy to report that drowning does kil...  [4, 2, 1, 2, 1, 1]   
2    """Let them praise the name of the LORD: for h...  [1, 1, 1, 1, 1, 2]   
3    """That was decades ago, Cecil! Anyway, we wan...  [1, 2, 1, 1, 1, 1]   
4    """This Jestxh idiot keeps riding me in dms, t...     [1, 3, 3, 3, 4]   
..                                                 ...                 ...   
621                         women are not dumb bitches        [1, 5, 2, 3]   
622  worst part about getting highlights is sitting...  [1, 3, 1, 1, 1, 1]   
623           yids are not as honest as the rest of us  [5, 5, 4, 5, 4, 1]   
624  you've got to be fucking joking, my room has l...  [3, 3, 1, 1, 2, 1]   
625  {@ClonedDeviant} « probably would have done th...     [1, 1, 1, 2, 1]   

                  toany                           annotatorMinority  \
0          [3, 3, 1, 2]    [{}, pansexual, disabled, trans, {}, {}]   
1    [4, 2, 1, 2, 2, 1]                    [{}, {}, {}, {}, {}, {}]   
2    [2, 1, 1, 1, 1, 3]        [{}, black; female, nan, {}, {}, {}]   
3    [1, 4, 1, 1, 1, 1]       [{}, black; female, nan, {}, {}, nan]   
4       [1, 2, 4, 4, 4]           [{}, NONE, No i do not, none, {}]   
..                  ...                                         ...   
621        [4, 5, 2, 3]  [pansexual, disabled, trans, {}, none, {}]   
622  [1, 5, 1, 1, 1, 1]                [{}, NONE, {}, {}, {}, none]   
623  [5, 5, 4, 5, 4, 1]           [nan, {}, {}, none, {}, bisexual]   
624  [2, 4, 1, 2, 3, 2]                  [{}, {}, {}, {}, none, {}]   
625     [1, 1, 1, 3, 1]                      [{}, {}, {}, {}, none]   

                                                  bert  \
0    [0.0047034588642418385, 0.11112377792596817, 0...   
1    [0.35706740617752075, 0.032704465091228485, 0....   
2    [-0.022276882082223892, 0.17076537013053894, 0...   
3    [0.07474598288536072, 0.24487990140914917, 0.2...   
4    [0.050774428993463516, 0.08187347650527954, 0....   
..                                                 ...   
621  [0.22457970678806305, 0.3840716481208801, -0.4...   
622  [0.1781517118215561, 0.02407737821340561, 0.13...   
623  [0.2983386516571045, 0.31099164485931396, -0.4...   
624  [0.6042274236679077, 0.4851360023021698, -0.24...   
625  [-0.044936660677194595, 0.07898200303316116, 0...   

                      annotatorPolitics      traditionalism  \
0                [-1.0, -1.0, 1.0, 0.5]        [3, 1, 5, 4]   
1    [-0.5, -0.5, 0.5, 0.5, -1.0, -1.0]  [4, 3, 4, 3, 1, 4]   
2    [-0.5, -1.0, 1.0, 0.0, -1.0, -0.5]  [3, 4, 3, 5, 1, 3]   
3     [0.0, -1.0, 1.0, 0.5, -0.5, -1.0]  [4, 4, 3, 4, 5, 5]   
4           [-1.0, 1.0, 0.5, 0.5, -1.0]     [5, 3, 5, 5, 5]   
..                                  ...                 ...   
621               [-1.0, 1.0, 0.5, 0.5]        [1, 5, 5, 5]   
622   [-1.0, 1.0, 0.5, 0.5, -0.5, -0.5]  [5, 3, 5, 2, 4, 3]   
623  [-0.5, -0.5, 0.5, 0.5, -0.5, -0.5]  [3, 5, 2, 5, 4, 3]   
624   [-0.5, -1.0, 0.0, 0.5, 0.0, -0.5]  [4, 5, 5, 3, 5, 5]   
625        [-0.5, -0.5, 0.5, 0.0, -0.5]     [5, 3, 2, 5, 3]   

                                  annotatorRace  \
0                  [black, black, white, white]   
1    [black, black, white, white, white, white]   
2    [black, black, white, white, white, white]   
3    [black, black, white, white, white, white]   
4           [black, black, white, white, white]   
..                                          ...   
621                [black, black, white, white]   
622  [black, black, white, white, white, white]   
623  [black, black, white, white, white, white]   
624  [black, black, white, white, white, white]   
625         [black, white, white, white, white]   

                             annotatorAge  \
0                [25.0, 35.0, 50.0, 25.0]   
1    [40.0, 30.0, 40.0, 30.0, 25.0, 25.0]   
2    [30.0, 40.0, 35.0, 40.0, 25.0, 45.0]   
3    [35.0, 

In [12]:
df = df.loc[
    :,
    [
        "tweet",
        "racism",
        "annotatorAge",
        "annotatorRace",
        "annotatorGender",
    ],
]
df

tweet              racism  \
0    " you ugly bitch!!!"- that seems to be my real...        [1, 1, 1, 4]   
1    """I am happy to report that drowning does kil...  [1, 1, 5, 5, 1, 1]   
2    """Let them praise the name of the LORD: for h...  [1, 1, 5, 1, 1, 1]   
3    """That was decades ago, Cecil! Anyway, we wan...  [1, 1, 5, 5, 1, 1]   
4    """This Jestxh idiot keeps riding me in dms, t...     [1, 3, 5, 1, 1]   
..                                                 ...                 ...   
621                         women are not dumb bitches        [1, 5, 1, 1]   
622  worst part about getting highlights is sitting...  [1, 5, 2, 3, 2, 1]   
623           yids are not as honest as the rest of us  [1, 1, 4, 1, 2, 1]   
624  you've got to be fucking joking, my room has l...  [1, 4, 1, 3, 3, 1]   
625  {@ClonedDeviant} « probably would have done th...     [1, 1, 4, 3, 1]   

                             annotatorAge  \
0                [25.0, 35.0, 50.0, 25.0]   
1    [40.0, 30.0, 40.0, 30.0, 25.0, 25.0]   
2    [30.0, 40.0, 35.0, 40.0, 25.0, 45.0]   
3    [35.0, 40.0, 35.0, 40.0, 35.0, 45.0]   
4          [45.0, 45.0, 45.0, 60.0, 40.0]   
..                                    ...   
621              [35.0, 30.0, 45.0, 65.0]   
622  [45.0, 45.0, 35.0, 35.0, 25.0, 50.0]   
623  [30.0, 30.0, 40.0, 60.0, 35.0, 40.0]   
624  [40.0, 25.0, 40.0, 35.0, 45.0, 35.0]   
625        [30.0, 30.0, 40.0, 30.0, 50.0]   

                                  annotatorRace  \
0                  [black, black, white, white]   
1    [black, black, white, white, white, white]   
2    [black, black, white, white, white, white]   
3    [black, black, white, white, white, white]   
4           [black, black, white, white, white]   
..                                          ...   
621                [black, black, white, white]   
622  [black, black, white, white, white, white]   
623  [black, black, white, white, white, white]   
624  [black, black, white, white, white, white]   
625         [black, white, white, white, white]   

                                annotatorGender  
0                      [man, man, woman, woman]  
1          [woman, woman, woman, man, man, man]  
2        [woman, woman, man, woman, man, woman]  
3        [man, woman, man, woman, woman, woman]  
4               [man, man, woman, woman, woman]  
..                                          ...  
621                    [man, woman, man, woman]  
622            [man, man, woman, man, man, man]  
623        [man, man, man, woman, woman, woman]  
624  [woman, woman, woman, woman, woman, woman]  
625                   [man, man, man, man, man]  

[626 rows x 5 columns]

Clear out NaNs from dataset

In [13]:
nan_age = df.annotatorAge.apply(lambda x: np.any(np.isnan(x)))
nan_race = df.annotatorRace.apply(lambda x: "na" in x)
nan_gender = df.annotatorGender.apply(lambda x: "na" in x)

nan_age.sum(), nan_race.sum(), nan_gender.sum()

(41, 41, 39)

In [14]:
def find_inconsistent_rows(df):
    inconsistent_rows = []

    for index, row in df.iterrows():
        list_lengths = [
            len(row[col]) for col in df.columns if isinstance(row[col], list)
        ]

        if len(set(list_lengths)) > 1:  # Check if lengths are inconsistent
            inconsistent_rows.append(index)

    return df.loc[inconsistent_rows]


find_inconsistent_rows(df)

Empty DataFrame
Columns: [tweet, racism, annotatorAge, annotatorRace, annotatorGender]
Index: []

In [15]:
df.annotatorRace = df.annotatorRace.apply(lambda x: None if ("na" in x) else x)
df.annotatorGender = df.annotatorGender.apply(
    lambda x: None if ("na" in x) else x
)
df.annotatorAge = df.annotatorAge.apply(
    lambda x: None if np.any([np.isnan(age) for age in x]) else x
)
df.annotatorAge = df.annotatorAge.apply(
    lambda x: [int(age) for age in x] if x is not None else None
)
df["random"] = df.racism.apply(
    lambda x: [np.random.randint(1, 3) for _ in range(len(x))]
)
df = df.dropna()
df

tweet              racism  \
0    " you ugly bitch!!!"- that seems to be my real...        [1, 1, 1, 4]   
1    """I am happy to report that drowning does kil...  [1, 1, 5, 5, 1, 1]   
2    """Let them praise the name of the LORD: for h...  [1, 1, 5, 1, 1, 1]   
3    """That was decades ago, Cecil! Anyway, we wan...  [1, 1, 5, 5, 1, 1]   
4    """This Jestxh idiot keeps riding me in dms, t...     [1, 3, 5, 1, 1]   
..                                                 ...                 ...   
621                         women are not dumb bitches        [1, 5, 1, 1]   
622  worst part about getting highlights is sitting...  [1, 5, 2, 3, 2, 1]   
623           yids are not as honest as the rest of us  [1, 1, 4, 1, 2, 1]   
624  you've got to be fucking joking, my room has l...  [1, 4, 1, 3, 3, 1]   
625  {@ClonedDeviant} « probably would have done th...     [1, 1, 4, 3, 1]   

                 annotatorAge                               annotatorRace  \
0            [25, 35, 50, 25]                [black, black, white, white]   
1    [40, 30, 40, 30, 25, 25]  [black, black, white, white, white, white]   
2    [30, 40, 35, 40, 25, 45]  [black, black, white, white, white, white]   
3    [35, 40, 35, 40, 35, 45]  [black, black, white, white, white, white]   
4        [45, 45, 45, 60, 40]         [black, black, white, white, white]   
..                        ...                                         ...   
621          [35, 30, 45, 65]                [black, black, white, white]   
622  [45, 45, 35, 35, 25, 50]  [black, black, white, white, white, white]   
623  [30, 30, 40, 60, 35, 40]  [black, black, white, white, white, white]   
624  [40, 25, 40, 35, 45, 35]  [black, black, white, white, white, white]   
625      [30, 30, 40, 30, 50]         [black, white, white, white, white]   

                                annotatorGender              random  
0                      [man, man, woman, woman]        [1, 1, 2, 2]  
1          [woman, woman, woman, man, man, man]  [2, 1, 1, 2, 2, 1]  
2        [woman, woman, man, woman, man, woman]  [1, 1, 1, 1, 1, 2]  
3        [man, woman, man, woman, woman, woman]  [2, 2, 1, 1, 1, 1]  
4               [man, man, woman, woman, woman]     [2, 2, 2, 1, 2]  
..                                          ...                 ...  
621                    [man, woman, man, woman]        [1, 1, 1, 1]  
622            [man, man, woman, man, man, man]  [1, 1, 1, 1, 2, 2]  
623        [man, man, man, woman, woman, woman]  [1, 2, 2, 2, 2, 1]  
624  [woman, woman, woman, woman, woman, woman]  [1, 1, 1, 1, 1, 1]  
625                   [man, man, man, man, man]     [2, 2, 2, 1, 2]  

[585 rows x 6 columns]

### Results

In [16]:
def extract_annotations_and_attributes(
    df: pd.DataFrame, value_col: str, feature_col: str, comment_key_col: str
) -> tuple[list, list]:
    all_annotations = []
    all_attributes = []
    all_keys = []

    for _, row in df.iterrows():
        values = row[value_col]
        factors = row[feature_col]
        key = row[comment_key_col]
        assert len(values) == len(
            factors
        ), f"values {values} (length {len(values)}) \n"
        f"have different length than factors{factors} (length {len(factors)})"
        all_annotations.extend(values)
        all_attributes.extend(factors)
        # extend the key for each value in the above extracted list
        all_keys.extend([row[comment_key_col]] * len(factors))

    return all_annotations, all_attributes, all_keys


def run_aposteriori(
    df: pd.DataFrame,
    value_col: str,
    feature_col: str,
    comment_key_col: str,
    bins: int = -1
) -> pd.Series:
    if bins == -1:
        bins = len(np.unique(df[value_col]))
    annotations, attributes, keys = extract_annotations_and_attributes(
        df=df,
        value_col=value_col,
        feature_col=feature_col,
        comment_key_col=comment_key_col,
    )

    stat = aposteriori.aposteriori_unimodality(
        annotations=annotations,
        factor_group=attributes,
        comment_group=keys,
        bins=bins,
    )
    return pd.Series(stat)

In [17]:
print(
    run_aposteriori(
        df,
        feature_col="annotatorAge",
        value_col="racism",
        comment_key_col="tweet",
        bins=10
    )
)
print(
    run_aposteriori(
        df,
        feature_col="annotatorGender",
        value_col="racism",
        comment_key_col="tweet",
    )
)
print(
    run_aposteriori(
        df,
        feature_col="annotatorRace",
        value_col="racism",
        comment_key_col="tweet",
    )
)
print(
    run_aposteriori(
        df,
        feature_col="random",
        value_col="racism",
        comment_key_col="tweet",
    )
)

18    5.904145e-01
25    1.660998e-30
30    2.328205e-09
35    1.000000e+00
40    1.290865e-04
45    2.357877e-01
50    2.026879e-21
55    2.333844e-01
60    1.116988e-08
65    2.664338e-08
dtype: float64
man          1.000000e+00
nonBinary    6.685300e-07
woman        1.000000e+00
dtype: float64


/home/dimits/miniconda3/envs/std/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1081: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/home/dimits/miniconda3/envs/std/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1081: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


black            6.434577e-01
hisp             1.649401e-09
middleEastern             NaN
other            8.797132e-02
white            1.000000e+00
dtype: float64
1    1.0
2    1.0
dtype: float64


## Synthetic data

In [18]:
def str_list_to_list(str_list: str) -> list[str]:
    ls = str_list[1:-1].split(",")
    ls = [x.strip() if x is not None else x for x in ls]
    return ls


syn_df = pd.read_csv("../data/synthetic.csv")
syn_df["comment_key"] = syn_df.message + syn_df.conv_id

syn_df.Toxicity = syn_df.Toxicity.apply(str_list_to_list).apply(
    lambda x: [int(tox) for tox in x]
)
syn_df.age_annot = syn_df.age_annot.apply(str_list_to_list)
syn_df.sex_annot = syn_df.sex_annot.apply(str_list_to_list)
syn_df.demographic_group_annot = syn_df.demographic_group_annot.apply(
    str_list_to_list
)
syn_df

conv_id  \
0     028f869d-cf12-444a-aefc-62f8b6294acd   
1     028f869d-cf12-444a-aefc-62f8b6294acd   
2     028f869d-cf12-444a-aefc-62f8b6294acd   
3     028f869d-cf12-444a-aefc-62f8b6294acd   
4     028f869d-cf12-444a-aefc-62f8b6294acd   
...                                    ...   
3543  fe858614-6571-43e7-badf-3cdcb38337b3   
3544  fe858614-6571-43e7-badf-3cdcb38337b3   
3545  fe858614-6571-43e7-badf-3cdcb38337b3   
3546  fe858614-6571-43e7-badf-3cdcb38337b3   
3547  fe858614-6571-43e7-badf-3cdcb38337b3   

                                                message  \
0     @FitnessJunkie07 Respect and honesty are unive...   
1     @FitnessJunkie07, I agree, but sometimes chang...   
2     @FitnessJunkie07, I completely agree that resp...   
3     @FitnessJunkie07, I understand where you're co...   
4     @LoyalVolunteer54 @FitnessJunkie07 It's import...   
...                                                 ...   
3543  @OptimisticTeacher59, you're cherry-picking st...   
3544  @OptimisticTeacher59, you're just as bad as @A...   
3545  @StoicSamurai99, you're missing the point. It'...   
3546  @StoicScholar73, thank you for your kind words...   
3547  Nothing in the Bible about Jesus saying anythi...   

                                                 intent  \
0     ['Neutral', 'Neutral', 'Neutral', 'Neutral', '...   
1     ['Neutral', 'Neutral', 'Neutral', 'Neutral', '...   
2     ['Community-oriented', 'Community-oriented', '...   
3     ['Community-oriented', 'Community-oriented', '...   
4     ['Moderator', 'Moderator', 'Moderator', 'Moder...   
...                                                 ...   
3543  ['Neutral', 'Neutral', 'Neutral', 'Neutral', '...   
3544  ['Troll', 'Troll', 'Troll', 'Troll', 'Troll', ...   
3545  ['Neutral', 'Neutral', 'Neutral', 'Neutral', '...   
3546  ['Neutral', 'Neutral', 'Neutral', 'Neutral', '...   
3547  ['Neutral', 'Neutral', 'Neutral', 'Neutral', '...   

                               Toxicity                   Argument Quality  \
0     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  [1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1]   
1     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]   
2     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  [1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2]   
3     [1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2]  [2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2]   
4     [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]   
...                                 ...                                ...   
3543  [2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]  [2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 1]   
3544  [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]  [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]   
3545  [2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1]  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]   
3546  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]   
3547  [3, 3, 3, 3, 2, 3, 4, 2, 3, 3, 3]  [4, 4, 4, 4, 3, 4, 5, 4, 4, 4, 4]   

                                               age_user  \
0     ['25', '25', '25', '25', '25', '25', '25', '25...   
1     ['27', '27', '27', '27', '27', '27', '27', '27...   
2     ['58', '58', '58', '58', '58', '58', '58', '58...   
3     ['58', '58', '58', '58', '58', '58', '58', '58...   
4     [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
...                                                 ...   
3543  ['40', '40', '40', '40', '40', '40', '40', '40...   
3544  ['35', '35', '35', '35', '35', '35', '35', '35...   
3545  ['40', '40', '40', '40', '40', '40', '40', '40...   
3546  ['43', '43', '43', '43', '43', '43', '43', '43...   
3547  ['43', '43', '43', '43', '43', '43', '43', '43...   

                                               sex_user  \
0     ['male', 'male', 'male', 'male', 'male', 'male...   
1     ['female', 'female', 'female', 'female', 'fema...   
2     ['female', 'female', 'female', 'female', 'fema...   
3     ['female', 'female', 'female', 'female', 'fema...   
4     [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
...                                                 ...   
3543 

In [19]:
find_inconsistent_rows(syn_df)

Empty DataFrame
Columns: [conv_id, message, intent, Toxicity, Argument Quality, age_user, sex_user, sexual_orientation_user, demographic_group_user, current_employment_user, education_level_user, age_annot, sex_annot, sexual_orientation_annot, demographic_group_annot, current_employment_annot, education_level_annot, comment_key]
Index: []

In [20]:
print(
    run_aposteriori(
        syn_df,
        feature_col="age_annot",
        value_col="Toxicity",
        comment_key_col="comment_key",
    )
)
print(
    run_aposteriori(
        syn_df,
        feature_col="sex_annot",
        value_col="Toxicity",
        comment_key_col="comment_key",
    )
)
print(
    run_aposteriori(
        syn_df,
        feature_col="demographic_group_annot",
        value_col="Toxicity",
        comment_key_col="comment_key",
    )
)

'21'    8.127109e-268
'27'    8.127109e-268
'28'    8.127109e-268
'32'    8.127109e-268
'38'    8.127109e-268
'40'    8.127109e-268
'45'    8.127109e-268
'50'    8.127109e-268
'55'    8.127109e-268
'60'    8.127109e-268
'65'    8.127109e-268
dtype: float64
'female'         4.836215e-10
'male'           1.215097e-02
'non-binary'    2.216484e-268
dtype: float64
'Arab'                6.649452e-268
'Black'               6.649452e-268
'Caucasian'            9.417159e-13
'Indigenous'          6.649452e-268
'Latino'              6.649452e-268
'Middle Eastern'      6.649452e-268
'Pacific Islander'    6.649452e-268
'South Asian'         6.649452e-268
'White'                1.058703e-11
dtype: float64


## Effect of bin size

In [22]:
for bins in range(3, 20):
    print(
        bins,
        run_aposteriori(
            df,
            feature_col="annotatorAge",
            value_col="racism",
            comment_key_col="tweet",
            bins=bins
        ),
    )

3 18    1.000000e+00
25    1.248173e-18
30    1.083296e-01
35    1.000000e+00
40    1.000000e+00
45    1.000000e+00
50    9.612009e-09
55    7.116560e-01
60    7.984303e-04
65    1.154033e-06
dtype: float64
4 18    5.904145e-01
25    3.571352e-28
30    2.448960e-05
35    1.000000e+00
40    7.123305e-02
45    1.000000e+00
50    1.183085e-17
55    3.774627e-01
60    3.858729e-08
65    6.219822e-07
dtype: float64
5 18    1.000000e+00
25    9.803728e-29
30    1.472915e-07
35    1.000000e+00
40    2.438705e-03
45    1.000000e+00
50    2.896304e-20
55    2.862292e-01
60    4.665271e-08
65    2.664338e-08
dtype: float64
6 18    5.904145e-01
25    4.181286e-30
30    5.652737e-08
35    1.000000e+00
40    9.586984e-04
45    6.212127e-01
50    9.796677e-21
55    2.862292e-01
60    2.112742e-08
65    2.664338e-08
dtype: float64
7 18    5.904145e-01
25    7.704192e-30
30    7.049811e-08
35    1.000000e+00
40    1.339200e-03
45    8.148805e-01
50    1.819146e-20
55    2.862292e-01
60    2.112742e-08